In [5]:
import re
import src.utils as utils
import numpy as np
import tensorflow as tf
import src.benchmark as benchmark
from tqdm.auto import tqdm

In [6]:
# import src.generate_vocabulary as gv
#
# gv.generate_vocabulary(['.\\data\\data'], ['py'], 25000)

In [7]:
# with open('.\\data\\python100k_train.txt') as f:
#     data_dirs = f.readlines()
#
#
# data_dirs = [re.sub(r'/', r'\\', ('.\\data\\'+d)[:-1]) for d in data_dirs]
# data_dirs = np.random.choice(data_dirs, size=50000, replace=False)
#
# tokenized_files = utils.tokenize_data(data_dirs)

In [8]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from src.model import Model

vocab_size = 20001 #CONST
embedding_dim = 32 #CONST
rnn_units = 128
batch_size = 128 #CONST
win_size = 5
model = Model(vocab_size, embedding_dim, rnn_units, batch_size, win_size, '..\\checkpoints\\2\\model.h5')
model.prepare_predictions('.\\vocabulary.voc', '..\\checkpoints\\2\\model.h5')
# print(model.get_prediction('', 100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 32)             640032    
_________________________________________________________________
lstm (LSTM)                  (1, None, 128)            82432     
_________________________________________________________________
dense (Dense)                (1, None, 20001)          2580129   
Total params: 3,302,593
Trainable params: 3,302,593
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
with open('..\\data\\my_eval.txt') as f:
    data_dirs = f.readlines()

np.random.seed(43)
data_dirs = [re.sub(r'/', r'\\', ('..\\data\\'+d)[:-1]) for d in data_dirs]
data_dirs = np.random.choice(data_dirs, size=9, replace=False)

proper_predictions, all_predictions = 0,0

In [10]:
progress = tqdm(data_dirs, position=0)

for file in progress:
    result = benchmark.single_benchmark(model, file, html=False)
    proper_predictions += result[0]
    all_predictions += result[1]

    if all_predictions != 0:
        progress.set_postfix({'accuracy' : proper_predictions/all_predictions})
print(f'total accuracy: {proper_predictions/all_predictions}')






total accuracy: 0.4636418269230769
